# 毫米波雷达配置参数合理性校验
1. 主要校验毫米波雷达需要的cfg文件及DCA采集板需要的cf.json文件的参数配置是否正确。
2. 参数的约束条件来自于IWR1843自身的器件特性，具体请参考IWR1843数据手册、mmwave SDK用户手册、chirp编程手册。
3. 若参数满足约束条件将以青色输出调试信息，若不满足则以紫色或黄色输出。
4. 需要注意的是，本程序约束条件并非完全准确，故特殊情况下即使参数全都满足约束条件，也有概率无法正常运行。

### 载入相关库

In [1]:
from color_log.clog import ColorLog, MODE_CONSOLE_LOG  # pip install py-color-log
import math
import json
from mmwave.dataloader import DCA1000
log = ColorLog(name='radar_param_logger', mode=MODE_CONSOLE_LOG)

### 读取配置文件
配置文件信息既可以通过DCA1000.read_config()方法从cfg文件读取，也可以手动输入ADC_PARAMS字典，其结构请参考修改配置文件章节。

In [2]:
# 读取毫米波雷达cfg配置文件
_,_,ADC_PARAMS,_=DCA1000.read_config("configFiles/xWR1843_profile_3D.cfg")
print(ADC_PARAMS)

# 读取DCA采集板cf.json配置文件
with open('configFiles/cf.json') as cf_json:
    cf_json_load = json.loads(cf_json.read())
    print(cf_json_load)

{'startFreq': 77, 'idleTime': 152.0, 'adc_valid_start_time': 7.0, 'rampEndTime': 61.0, 'freq_slope': 65.0, 'txStartTime': 1.0, 'samples': 512, 'sample_rate': 9562, 'chirps': 32, 'frame_periodicity': 1000.0, 'rx': 4, 'tx': 3, 'IQ': 2, 'bytes': 2}
{'DCA1000Config': {'dataLoggingMode': 'raw', 'dataTransferMode': 'LVDSCapture', 'dataCaptureMode': 'ethernetStream', 'lvdsMode': 1, 'dataFormatMode': 3, 'packetDelay_us': 5}}


### 计算约束条件
若参数满足约束条件将以青色输出调试信息，若不满足则以紫色或黄色输出。以黑色输出的为计算出的距离、速度分辨率等信息。

In [3]:
# 约束条件主要参考Programming Chirp Parameters in TI Radar Devices (Rev. A).pdf

# 计算线性调频chirp脉冲的斜率是否满足约束条件：freqSlope<=100MHz/us
if (ADC_PARAMS['startFreq'] >= 76):
    CLI_FREQ_SCALE_FACTOR = 3.6 # 77GHz
else:
    CLI_FREQ_SCALE_FACTOR = 2.7 # 60GHz
mmwFreqSlopeConst = math.trunc(ADC_PARAMS['freq_slope'] * (1<<26) / CLI_FREQ_SCALE_FACTOR )
freqSlopeConst_actual = mmwFreqSlopeConst * (CLI_FREQ_SCALE_FACTOR / (1<<26))
logstr='mmwFreqSlopeConst: %d,freqSlopeConst_actual: %f(MHz/us) must <= 100'%(mmwFreqSlopeConst,freqSlopeConst_actual)
if(freqSlopeConst_actual<=100):
    log.info(logstr)
else:
    log.error(logstr)

# 计算活动帧周期（Tf=N*Tc）是否满足约束条件：active_frame_time<ADC_PARAMS['frame_periodicity']
chirp_repetition_period=ADC_PARAMS['tx']*(ADC_PARAMS['idleTime']+ADC_PARAMS['rampEndTime'])# Tc（us），chirp脉冲间隔周期
log.debug('chirp repetition period:%.2f(us)->max single tx chirp freq:%.2f(kHz)'%(chirp_repetition_period,1e3/chirp_repetition_period))
active_frame_time = ADC_PARAMS['chirps']*chirp_repetition_period/1e3 # Tf=N*Tc（ms）
logstr='active frame time: %.3f(ms) must < frame periodicity: %.3f(ms)'%(active_frame_time,ADC_PARAMS['frame_periodicity'])
if(active_frame_time<ADC_PARAMS['frame_periodicity']):
    log.info(logstr)
    all_tx_chirp_freq=ADC_PARAMS['tx']*ADC_PARAMS['chirps']/ADC_PARAMS['frame_periodicity']
    log.debug('All tx chirp repetition period:%.2f(us)->All tx chirp freq:%.2f(kHz)'%(1e3/all_tx_chirp_freq,all_tx_chirp_freq))
else:
    log.error(logstr)

# 计算采样带宽是否满足约束条件：采样带宽<有效脉冲斜坡带宽（由chirp脉冲斜坡持续时间及斜率决定）
adc_sample_time = 1e3*ADC_PARAMS['samples']/ADC_PARAMS['sample_rate']
max_adc_sample_time = ADC_PARAMS['rampEndTime']-ADC_PARAMS['adc_valid_start_time']-0.15 #不减0.15会造成末尾采样点失真
log.debug('adc start time:%.2f(us) adc sample time:%.2f(us) max adc sample time:%.2f(us) ramp end time:%.2f(us)'%(ADC_PARAMS['adc_valid_start_time'],adc_sample_time,max_adc_sample_time,ADC_PARAMS['rampEndTime']))
adc_sample_sweep_bandwidth = freqSlopeConst_actual * 1e6 * adc_sample_time
max_adc_sweep_bandwidth = freqSlopeConst_actual * 1e6 * max_adc_sample_time
logstr=('adc_sample_sweep_bandwidth:%.2f(MHz) must < max_adc_sweep_bandwidth:%.2f(MHz)'%(adc_sample_sweep_bandwidth/1e6,max_adc_sweep_bandwidth/1e6))
if(adc_sample_sweep_bandwidth < max_adc_sweep_bandwidth):
    log.info(logstr)
else:
    log.error(logstr)

# 计算总脉冲斜坡带宽是否满足约束条件：总脉冲斜坡带宽<4GHz
total_ramp_sweep_bandwidth = freqSlopeConst_actual * 1e6 * ADC_PARAMS['rampEndTime']
logstr=('total_ramp_sweep_bandwidth:%.2f(MHz) must <= 4000(MHz)'%(total_ramp_sweep_bandwidth/1e6))
if(total_ramp_sweep_bandwidth/1e6<=4000):
    log.info(logstr)
else:
    log.error(logstr)

# 计算两脉冲间隔空闲时间是否满足约束条件：ADC_PARAMS['idleTime']>=synthesizerRampDownTime
if(total_ramp_sweep_bandwidth/1e6<1000):
    synthesizerRampDownTime=2
elif(total_ramp_sweep_bandwidth/1e6<2000):
    synthesizerRampDownTime=3.5
elif(total_ramp_sweep_bandwidth/1e6<3000):
    synthesizerRampDownTime=5
else:
    synthesizerRampDownTime=7
logstr='idle time: %.3f(us) must >= Synthesizer Ramp Down Time: %.3f(us)'%(ADC_PARAMS['idleTime'],synthesizerRampDownTime)
if(ADC_PARAMS['idleTime']>=synthesizerRampDownTime):
    log.info(logstr)
else:
    log.error(logstr)

# 计算并显示距离、速度分辨率等参数
midFreq=ADC_PARAMS['startFreq']*1e9+(ADC_PARAMS['adc_valid_start_time']+adc_sample_time/2)*freqSlopeConst_actual*1e6
midFreqWaveLenth=3e8/midFreq

numDopplerBins = 2**math.ceil(math.log2(ADC_PARAMS['chirps']))
numRangeBins = 2**math.ceil(math.log2(ADC_PARAMS['samples']))

rangeResolutionMeters = 3e8 / (2 * adc_sample_sweep_bandwidth)
rangeIdxToMeters = (3e8 * ADC_PARAMS['sample_rate'] * 1e3) / (2 * freqSlopeConst_actual * 1e12 * numRangeBins)
dopplerResolutionMps = midFreqWaveLenth / (2 * numDopplerBins * chirp_repetition_period*1e-6)
maxRange = (300 * 0.8 * ADC_PARAMS['sample_rate']) / (2 * freqSlopeConst_actual * 1e3)
# maxRange = 0.8 * rangeResolutionMeters * ADC_PARAMS['samples']
maxVelocity = midFreqWaveLenth / (4 * chirp_repetition_period*1e-6)

log.debug('max range:%.2f(m)'%maxRange)
log.debug('range resolution:%.4f(m),range interbin resolution:%.4f(m)'%(rangeResolutionMeters,rangeIdxToMeters))
log.debug('max velocity:%.2f(m/s)'%maxVelocity)
log.debug('velocity resolution:%.2f(m/s)'%(dopplerResolutionMps))

# 计算混频输出的最大拍频信号IF带宽是否满足约束条件：maximum_beat_frequency<=10(MHz)
maximum_beat_frequency = 2e6*freqSlopeConst_actual*maxRange/3e8
logstr=('max beat frequency:%.2f(MHz) must <= max I/F bandwith 10(MHz)'%maximum_beat_frequency)
if(maximum_beat_frequency<=10):
    log.info(logstr)
else:
    log.error(logstr)

# 计算片内DSP需要的radar cube内存大小是否满足约束条件：radar_cube_size<=1024(KB)
radar_cube_size = numRangeBins*ADC_PARAMS['chirps']*ADC_PARAMS['tx']*ADC_PARAMS['rx']*4/1024
logstr=('radar cube size:%.2f(KB) must <= 1024(KB)'%radar_cube_size)
if(radar_cube_size<=1024):
    log.info(logstr)
else:
    log.error(logstr)

# 计算原始IQ数据量是否满足约束条件：LVDS Data Size Per Chirp <= max Send Bytes Per Chirp
LVDSDataSizePerChirp = ADC_PARAMS['samples']*ADC_PARAMS['rx']*ADC_PARAMS['IQ']*ADC_PARAMS['bytes']+52
LVDSDataSizePerChirp = math.ceil(LVDSDataSizePerChirp/256)*256
nlane = 2
B = 600
maxSendBytesPerChirp = (ADC_PARAMS['idleTime']+ADC_PARAMS['rampEndTime'])*nlane*B/8
logstr=("LVDS Data Size Per Chirp:%d(Bytes) must <= max Send Bytes Per Chirp:%d(Bytes)"%(LVDSDataSizePerChirp,maxSendBytesPerChirp))
if(LVDSDataSizePerChirp<=maxSendBytesPerChirp):
    log.info(logstr)
else:
    log.error(logstr)

# 计算FPGA吞吐量（与延时有关）是否满足约束条件：FPGA Packet Delay <= min required Packet Delay
BYTES_IN_PACKET = 1456  # Data in payload per packet from FPGA
BYTES_OF_PACKET = 1466  # payload size per packet from FPGA
UDP_PACKET_OVERHEAD = 8
IP_OVERHEAD     = 20
ETH_OVERHEAD    = 14
SizeInMBperSec = ADC_PARAMS['samples']*ADC_PARAMS['rx']*ADC_PARAMS['IQ']*ADC_PARAMS['bytes']*ADC_PARAMS['tx']*ADC_PARAMS['chirps']*(1e-3/ADC_PARAMS['frame_periodicity'])
overheadRatio = (BYTES_OF_PACKET+UDP_PACKET_OVERHEAD+IP_OVERHEAD+ETH_OVERHEAD)/BYTES_IN_PACKET
FPGA_throughput = 8*SizeInMBperSec*overheadRatio
log.debug('FPGA throughput:%.2f(Mbps)'%(FPGA_throughput))
minPacketDelay = 138.57 * math.exp(-FPGA_throughput / 179.157) + 2.73 # 根据DCA1000EVA手册数据拟合
logstr=("FPGA Packet Delay:%d(us) should <= min required Packet Delay:%.2f(us)"%(cf_json_load['DCA1000Config']['packetDelay_us'],minPacketDelay))
if(cf_json_load['DCA1000Config']['packetDelay_us']<=minPacketDelay):
    log.info(logstr)
else:
    log.warn(logstr)

print('current params:',ADC_PARAMS)

2023-04-08 23:51:43 [INFO] - mmwFreqSlopeConst: 1211687822,freqSlopeConst_actual: 65.000000(MHz/us) must <= 100
2023-04-08 23:51:43 [DEBUG] - chirp repetition period:639.00(us)->max single tx chirp freq:1.56(kHz)
2023-04-08 23:51:43 [INFO] - active frame time: 20.448(ms) must < frame periodicity: 1000.000(ms)
2023-04-08 23:51:43 [DEBUG] - All tx chirp repetition period:10416.67(us)->All tx chirp freq:0.10(kHz)
2023-04-08 23:51:43 [DEBUG] - adc start time:7.00(us) adc sample time:53.55(us) max adc sample time:53.85(us) ramp end time:61.00(us)
2023-04-08 23:51:43 [INFO] - adc_sample_sweep_bandwidth:3480.44(MHz) must < max_adc_sweep_bandwidth:3500.25(MHz)
2023-04-08 23:51:43 [INFO] - total_ramp_sweep_bandwidth:3965.00(MHz) must <= 4000(MHz)
2023-04-08 23:51:43 [INFO] - idle time: 152.000(us) must >= Synthesizer Ramp Down Time: 7.000(us)
2023-04-08 23:51:43 [DEBUG] - max range:17.65(m)
2023-04-08 23:51:43 [DEBUG] - range resolution:0.0431(m),range interbin resolution:0.0431(m)
2023-04-08 2

current params: {'startFreq': 77, 'idleTime': 152.0, 'adc_valid_start_time': 7.0, 'rampEndTime': 61.0, 'freq_slope': 65.0, 'txStartTime': 1.0, 'samples': 512, 'sample_rate': 9562, 'chirps': 32, 'frame_periodicity': 1000.0, 'rx': 4, 'tx': 3, 'IQ': 2, 'bytes': 2}


### 修改配置文件参数
可以在此处试着修改一些参数并重新运行“计算约束条件”程序块校验参数的合理性

In [105]:
#在这里修改参数后直接运行上边的程序块看效果
ADC_PARAMS={
    'startFreq': 77,             # 对应 profileCfg 中第 2 个参数
    'idleTime': 8.0,             # 对应 profileCfg 中第 3 个参数 
    'adc_valid_start_time': 7.0, # 对应 profileCfg 中第 4 个参数 
    'rampEndTime': 50,           # 对应 profileCfg 中第 5 个参数 
    'freq_slope': 80,            # 对应 profileCfg 中第 8 个参数 
    'txStartTime': 1,            # 对应 profileCfg 中第 9 个参数 
    'samples': 128,              # 对应 profileCfg 中第 10个参数 
    'sample_rate': 12500,        # 对应 profileCfg 中第 11个参数 

    'chirps': 128,               # 对应   frameCfg 中第 3 个参数
    'frame_periodicity': 10,     # 对应   frameCfg 中第 5 个参数
    'rx': 4, 
    'tx': 1, 
    'IQ': 2, 
    'bytes': 2, 
}

### 测试
测试ColorLog模块的功能

In [1]:
from color_log.clog import ColorLog
log = ColorLog(name='radar_param_logger', mode=MODE_CONSOLE_LOG)

log.debug('这是 DEBUG 日志，白色')
log.info('这是 INFO 日志，青绿')
log.warn('这是 WARN 日志，黄色')
log.error('这是 ERROR 日志，紫色')
try :
    a = 1 / 0
except :
    log.critical('这是 CRITICAL 日志，红色')

2022-07-02 13:36:52 [DEBUG] - 这是 DEBUG 日志，白色
2022-07-02 13:36:52 [INFO] - 这是 INFO 日志，青绿
2022-07-02 13:36:52 [WARNING] - 这是 WARN 日志，黄色
2022-07-02 13:36:52 [ERROR] - 这是 ERROR 日志，紫色
2022-07-02 13:36:52 [CRITICAL] - 这是 CRITICAL 日志，红色
2022-07-02 13:36:52 [CRITICAL] - Traceback (most recent call last):
  File "C:\Users\gwf\AppData\Local\Temp\ipykernel_14708\2271216170.py", line 8, in <cell line: 7>
    a = 1 / 0
ZeroDivisionError: division by zero

